In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

import pandas as pd

In [3]:
  query = f""" *
  from "RISKACCESS"."riskDebtAccounts"
fetch first 3 rows only
"""
data= export_from_RISKCUSTOM(query)
# data.to_excel('USD-BRL_2022-2024.xlsx')
data

DatabaseError: Execution failed on sql ' *
from "RISKACCESS"."riskDebtAccounts"
fetch first 3 rows only
': ORA-00900: invalid SQL statement

In [2]:
data = export_from_RISKCUSTOM(f"""select * from "RISKACCESS"."mdgSalesUnits" """)
# data.to_excel('SalesUnits.xlsx')
data.head(2)

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:98: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


,id,division,instance,registryCountry,registryIdentifier,OEBSIntegrated,CAMIntegrated,PTVIntegrated,region,company_id,...,ocpSegment,affiliatedHolding,businessSegmentDetailed,quantumId,isInactive,holdingManagement,isFixedCost,trpMarket,functionalCurrency,ifrsCurrency
0,RUTLI,SALES,SAP,RU,None,0.0,0.0,0.0,Russia & CIS,5411073.0,...,RUFERT,None,RUSSIA,None,None,EUROCHEM,N,None,RUB,RUB
1,7Q00,None,SAP,PL,None,NaN,NaN,NaN,SUEK AG,NaN,...,EX SUEK AG,None,SUEK AG+,None,None,SUEK,None,None,None,None


In [4]:
data = export_from_RISKCUSTOM(f"""DESCRIBE "RISKACCESS"."mdgSalesUnits" """)
data

DatabaseError: Execution failed on sql 'DESCRIBE "RISKACCESS"."mdgSalesUnits" ': ORA-00900: invalid SQL statement

In [7]:
data_M = pd.read_excel('buNameToIdMapping_2024_08.xlsx',header=5,usecols=[2,3])
data_M['Source'] = 'M'
data_M

,buName,salesUnitId,Source
0,АО «МХК «ЕВРОХИМ»- ФИЛИАЛ В БЕЛАРУСИ-XCSAL,BYEMI,M
1,АО «ПРОМТРАНС»,4G00,M
2,"АО «СУЭК-КУЗБАСС""",5N00,M
3,АО АБАКАНСКАЯ ТЭЦ,VH00,M
4,АО БАРНАУЛЬСКАЯ ГЕНЕРАЦИЯ,W100,M
...,...,...,...
583,"Западно-Сибирский филиал ООО ""Сибирская генери...",1R00,M
584,EUROCHEM COMERCIO DE PRODUTOS QUIMI LTDA,BRCPQ,M
585,PT. Suek Trading Indonesia,7N00,M
586,TerraBrown AG,7100,M


In [7]:
import pandas as pd
data_A = pd.read_csv('Mapping.csv')
# data_A['Source'] = 'A'
data_A[data_A.id=='CHTRD']
data_A.id = data_A.id.replace({'CHTRD':'CHEAG'})
data_A[data_A.id=='CHTRD']


,CompName,id


In [9]:
data_A.to_csv('Mapping.csv', index=False)

In [15]:
data_M[data_M.duplicated(subset='buName', keep=False)]
data_A[data_A.duplicated(subset='CompName', keep=False)]

,CompName,id,Source
94,"АО ""КФС-Транс""",1W00,A
513,IEFIN,IEFIN,A
520,RUEMO,RUEMO,A
558,"АО ""КФС-Транс""",BCS-1Q01,A
666,RUEMO,RUEMO,A
774,IEFIN,IEFIN,A


In [19]:
data_A_new = data_A.drop_duplicates(subset='CompName', keep='first').reset_index(drop=True)

In [22]:
data_merge = data_M.merge(data_A_new, how='outer', left_on='buName', right_on='CompName')
data_merge

,buName,salesUnitId,Source_x,CompName,id,Source_y
0,"(УДАЛЕН)АО ""РАЗРЕЗ НАЗАРОВСКИЙ""",2N00,M,NaN,NaN,NaN
1,NaN,NaN,NaN,"(удален) ООО ""Черновский РМЗ""",3900,A
2,NaN,NaN,NaN,"(удален)АО ""Разрез Назаровский""",2N00,A
3,NaN,NaN,NaN,"(удален)ООО ""Урал-ремстройсервис""",RURBZ,A
4,NaN,NaN,NaN,1100,1100,A
...,...,...,...,...,...,...
1289,NaN,NaN,NaN,"ЧОУ ДПО ""Энергоцентр""",Y600,A
1290,"ЧУ ДПО КУЦ ""ЭНЕРГЕТИК""",VD00,M,NaN,NaN,NaN
1291,NaN,NaN,NaN,"ЧУ ДПО КУЦ ""Энергетик""",VD00,A
1292,NaN,NaN,NaN,"филиал АО ""Барнаульская генерация"" ""Барнаульск...",W500,A


In [29]:
data_1 = data_merge[data_merge.buName.isna()]
data_2 = data_merge[data_merge.CompName.isna()]
# data_merge[data_merge.duplicated('buName', keep=False)]

In [31]:
data_1.loc[:, 'CompName':].to_excel('buNameToIdMapping_2024_08_VS_Mapping.xlsx')

In [35]:
data_2.loc[:,:'salesUnitId'].to_csv('Mapping.csv', header=False, mode='a', index=False)

In [52]:
data_new = pd.read_csv('Mapping.csv')
data_new
data_A_fresh = data_new.drop_duplicates('CompName', keep='first')

In [53]:
data_A_fresh

,CompName,id
0,"ООО ""ПГ ""Фосфорит""",RUFSF
1,"АО ""Черногорский РМЗ""",4700
2,Eurochem Group UAE Branch,AEEAG
3,"АО ""Кабактинское""",4800
4,"ООО ""АЛТАЙ-СУЭК""",4900
...,...,...
1292,"ФИЛИАЛ АО ""НАК ""АЗОТ"" НОВОМОСКОВСКАЯ ГРЭС",RUNAK
1293,ФИЛИАЛ ТБТ,RUTBN
1294,"ФОНД ""СУЭК-РЕГИОНАМ""",F199
1295,"ЧОУ ДПО ""ЭНЕРГОЦЕНТР""",Y600


In [54]:
data_A_fresh.loc[:,].to_csv('Mapping.csv', index=False)